# Assignment 5

## Assignment 5.1

Given are the following points A(−0.5;1), B(−1;−1.5), C(−1.5;1.5), D(1.5;−0.5) and E(0.5;−0.5) as shown below. The goal is to learn a decision function $f$ using Linear Learning Machines (LLM) that separates A and E from the other points.

<img src="5_figure.png" width="400">

a) Compute the kernel matrix for $K(x_1, x_2) = \langle x_1, x_2  \rangle ^2 $.

In this section, we need to battle not only the exercise but also very unclear notation. With regards to the kernel matrix, what we are asked to do is to compute the square of the dot product between the vectors $x_i$ which are made up of 2 values, confusingly called $x$ and $y$ in the figure. For the purposes of clarity, I will refrain from using $x$ to refer to the abscissa feature and instead $x_i$ will signify the entire vector made up of 2 values.

In [30]:
import numpy as np

# Pre-allocate an empty array 
K = np.empty((5, 5))

# Initialise all data points 
A = [-0.5, 1, 1]; B = [-1, -1.5, -1]; C = [-1.5, 1.5, -1]; D = [1.5, -0.5, -1]; E = [0.5, -0.5, 1]
data = [A, B, C, D, E]

for i in range(5):
    for j in range(5):
        K[i, j] = np.dot(data[i][:2], data[j][:2])**2
K

array([[ 1.5625,  1.    ,  5.0625,  1.5625,  0.5625],
       [ 1.    , 10.5625,  0.5625,  0.5625,  0.0625],
       [ 5.0625,  0.5625, 20.25  ,  9.    ,  2.25  ],
       [ 1.5625,  0.5625,  9.    ,  6.25  ,  1.    ],
       [ 0.5625,  0.0625,  2.25  ,  1.    ,  0.25  ]])

b) Apply the perceptron update rule in dual representation (see below) to all five
data points using the kernel from (a). Start with α = $\vec{0}$, $b = 0$ and repeat the
updating until all data points can be correctly classified:

$$
\forall i : y_i \left( \sum_{j=1}^n \alpha_j y_j \langle x_1, x_2  \rangle ^2 + b \right) \leq 0  \Rightarrow \alpha_i = \alpha_i + 1; b = b + y_i (\underset{j}{\max} || x_j ||)^2
$$

Here is more bad notation, to the extent the notation becomes atrocious. Firstly, while not necessarily confusing, it is not explained that the $i$ and $j$ indices correspond to the same loop executed twice, we are doing a pair-wise operation in other words. Secondly, the dot product, indicated by the angled brackets, actually refers to the dot product between $x_i$ and $x_j$. Lastly, the change in $b$ is particularly poorly given because it's a single value that does not change from one iteration over $i$ to another, it's always going to be the squared norm of point C (4.5) multiplied by some factor. 

In [67]:
# Initialise the Lagrangian multiplier vector
lagrange_vec = np.array([0]*5)

# Initialise b
b = 0

# Create a flag
miss_clas = True 

while miss_clas:
    miss_clas = False
    
    for i in range(5):
        clsf = data[i][-1] * (sum([lagrange_vec[j]*data[j][-1]*K[i, j] for j in range(5)]) + b)
    
        if clsf <= 0:
            lagrange_vec[i] += 1
            b += 4.5 * data[i][-1]
            miss_clas = True 
            
            print(f"The decision value {clsf} is below (or equal to) 0, so new alpha values are {lagrange_vec} and b is {b}")

The decision value 0.0 is below (or equal) 0, so new alpha values are [1 0 0 0 0] and b is 4.5
The decision value -5.5 is below (or equal) 0, so new alpha values are [1 1 0 0 0] and b is 0.0
The decision value -4.5 is below (or equal) 0, so new alpha values are [1 1 1 0 0] and b is -4.5
The decision value -6.25 is below (or equal) 0, so new alpha values are [1 1 1 0 1] and b is 0.0
The decision value -3.9375 is below (or equal) 0, so new alpha values are [2 1 1 0 1] and b is 4.5


c) Classify the point X(-1;0) using the learned hyperplane.

In [74]:
clsf_np = data[i][-1] * (sum([lagrange_vec[j]*data[j][-1]*np.dot(data[j][:2], [-1, 0])** 2 for j in range(5)]) + b)
print(f"The new point is classified as a cross (1) since its decision values is {clsf_np} which is larger than zero") 

The new point is classified as a cross (1) since its decision values is 2.0 which is larger than zero


## Assignment 5.2

What is the main idea behind linear Support Vector Machines (SVMs)? Illustrate your
explanation by drawing a figure. What equations are used in SVMs? How can a separating
hyperplane with a maximal margin be found?

The basic idea of SVMs is to find a hyperplane by virtue of implicitly mapping data to a space where they can be linearly separated using kernels. An important property of SVMs is that the determination of the model parameters corresponds to a convex optimisation problem, thus rendering any local solution a global optimum. 

The central equation of SVMs is centred around the decision function $f(x) = \langle w, x \rangle + b = \sum_i \alpha_i y_i \langle x_i, x \rangle+b$

Below is a figure illustrating the implicit mapping to a new space with the associated kernel, newly rendering the problem linearly separable


<img src="5_2_figure.png" width="600">


In order to find the hyperplane with the maximal margin, we seek to minimise the size of the vector $w$ as per the figure below. Note the inconsistent notation with the bias term being subtracted rather than added, though the meaning isn't changed as the unary negation doesn't affect the linearity of the separation. 

<img src="5_3_figure.png" width="400">

The minimisation itself is done in the context of constrained optimisation using Lagrange multipliers.

## Assignment 5.3

How is the optimization problem of a Support Vector Machine modified to handle not
linearly separable data?

The central idea of SVMs is to introduce kernels which implicitly map the data into a new space where they are linearly separable. We can also use slack variables for a soft margin which allows for misclassification. 

## Assignment 5.4

Present a kernel that was not described in the course. You should be able to describe how it
is computed and in which scenarios it can be used. Look for articles describing this kernel.
Examples of such kernels are string, tree or graph kernels or kernels for bioinformatics.